<a href="https://colab.research.google.com/github/max-broekhuis/workshop-consultancy-day/blob/main/code_ec_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm

# Inladen Data



In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=1mlqgfqfBmp9Att1KHQZADannFgAqt7l6' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt
!curl -L -b cookies.txt -o 'EC_Energy_case.xlsx' 'https://docs.google.com/uc?export=download&id=1mlqgfqfBmp9Att1KHQZADannFgAqt7l6&confirm='$(<confirm.txt)
!rm -f confirm.txt cookies.txt

In [ ]:
data = pd.read_excel("EC_Energy_case.xlsx")
data = data.assign(totaal_zonnepanelen = lambda x: x.aantal_installaties_bij_woningen*x.aantal_zonnepanelen_per_installatie)

# Analyse data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data["gemeentenaam"].nunique()

In [ ]:
data["soort_regio"].value_counts()

In [ ]:
 (
    data
    .groupby(["gemeentenaam"])
    ["totaal_aantal_laadpalen"]
    .mean()
    .sort_values(ascending = False)
    .head(500)
)


In [ ]:
(
    data
    .groupby(["gemeentenaam"])
    ["totaal_zonnepanelen"]
    .mean()
    .sort_values(ascending = False)
    .head(500)
)

# Regressie

Omdat EC Energy intern niet veel werknemers heeft met verstand van Data Science is er een staigaire gevraagd om een opzet te maken van een model.

Deze staigaire heeft de data gefiltered op een aantal interessante variabelen. Daarna zijn deze variabelen door een lineaire regressie gehaald. Op basis van de resultaten is EC Energy enthousiast, maar geeft de staigaire aan dat er problemen zijn met deze analyse en een verdieping noodzakelijk is. Daarom heeft EC Energy meerdere consultancy partijen gevraag een business case op te stellen.

In [ ]:
data['EV_totaal'] = data['personenautos_overige_brandstof'] * 0.2333
X = (data[
          [
           "EV_totaal",
           "gemiddelde_huishoudensgrootte",
           "gemiddelde_woningwaarde",
           "totaal_aantal_laadpalen",
           "huishoudens_totaal",
           "k_0_tot_15_jaar",
           "k_15_tot_25_jaar",
           "k_25_tot_45_jaar",
           "k_45_tot_65_jaar",
           "k_65_jaar_of_ouder",
           "eenpersoonshuishoudens",
           "huishoudens_zonder_kinderen",
           "huishoudens_met_kinderen",
           "aantal_inkomensontvangers",
           "gemiddeld_inkomen_per_inkomensontvanger",
           "gemiddeld_inkomen_per_inwoner",
           "k_40_huishoudens_met_laagste_inkomen",
           "k_20_huishoudens_met_hoogste_inkomen"
           ]
          ])
X.dropna(inplace =True)

In [130]:
x = X.drop("totaal_aantal_laadpalen", axis=1)
y = X["totaal_aantal_laadpalen"]
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                               OLS Regression Results                              
Dep. Variable:     totaal_aantal_laadpalen   R-squared:                       0.671
Model:                                 OLS   Adj. R-squared:                  0.669
Method:                      Least Squares   F-statistic:                     343.2
Date:                     Mon, 14 Jun 2021   Prob (F-statistic):               0.00
Time:                             14:21:17   Log-Likelihood:                -10766.
No. Observations:                     2874   AIC:                         2.157e+04
Df Residuals:                         2856   BIC:                         2.167e+04
Df Model:                               17                                         
Covariance Type:                 nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

# Vraagstelling

Na het zien van deze regressie is EC Energy benieuwd wat er nog meer mogelijk is met deze data. Daarom hebben ze jou gevraagd om een business case op te zetten. Hierin wilt EC Energy een duidelijk advies waar ze hun salesstrategie op moeten toespitsen. Inhoudelijk hebben ze weinig verstand van Data Science en daarom laten ze jou vrij om iedere techniek te gebruiken waarvan jij denkt dat het het meeste resultaat gaat opleveren.

EC Energy gaat jou beoordelen op hoe begrijpbaar jouw salesstrategie is (50%), hoe goed je model/tool werkt (20%) en hoe dit zich verhoudt tot je concurrenten (30%).